In [2]:
import requests
import pandas as pd
import os
import sys
sys.path.insert(0, 'src')
from api_keys import GOOGLE_API_KEY

os.makedirs('data/images', exist_ok=True)

# Test with just one LSOA
test = pd.read_csv('data/test_lsoas.csv')
row = test.iloc[0]

url = (
    f"https://maps.googleapis.com/maps/api/streetview"
    f"?size=640x640"
    f"&location={row['lat']},{row['lon']}"
    f"&heading=0"
    f"&pitch=0"
    f"&fov=90"
    f"&key={GOOGLE_API_KEY}"
)

response = requests.get(url)
print(f"Status code: {response.status_code}")
print(f"Image size: {len(response.content)} bytes")

if len(response.content) > 10000:
    with open('data/images/test_image.jpg', 'wb') as f:
        f.write(response.content)
    print("Success! Open data/images/test_image.jpg to see it")
else:
    print("Failed - image too small, likely no Street View coverage here")

Status code: 200
Image size: 63572 bytes
Success! Open data/images/test_image.jpg to see it


In [1]:
import requests
import pandas as pd
import os
import time
import sys
sys.path.insert(0, 'src')
from api_keys import GOOGLE_API_KEY

test = pd.read_csv('data/test_lsoas.csv')
os.makedirs('data/images', exist_ok=True)

headings = [0, 90, 180, 270]
total_success = 0
total_failed = 0

for idx, row in test.iterrows():
    lsoa_code = row['LSOA11CD']
    
    for heading in headings:
        filename = f"data/images/{lsoa_code}_{heading}.jpg"
        
        # Skip if already downloaded
        if os.path.exists(filename):
            total_success += 1
            continue
        
        url = (
            f"https://maps.googleapis.com/maps/api/streetview"
            f"?size=640x640"
            f"&location={row['lat']},{row['lon']}"
            f"&heading={heading}"
            f"&pitch=0"
            f"&fov=90"
            f"&key={GOOGLE_API_KEY}"
        )
        
        response = requests.get(url)
        
        if response.status_code == 200 and len(response.content) > 10000:
            with open(filename, 'wb') as f:
                f.write(response.content)
            total_success += 1
        else:
            total_failed += 1
        
        time.sleep(0.3)
    
    # Print progress every 50 LSOAs
    if (idx + 1) % 50 == 0:
        print(f"Progress: {idx + 1}/500 LSOAs | Success: {total_success} | Failed: {total_failed}")

print(f"\nDone! {total_success} downloaded, {total_failed} failed")

Progress: 50/500 LSOAs | Success: 180 | Failed: 20
Progress: 100/500 LSOAs | Success: 356 | Failed: 44
Progress: 150/500 LSOAs | Success: 524 | Failed: 76
Progress: 200/500 LSOAs | Success: 700 | Failed: 100
Progress: 250/500 LSOAs | Success: 880 | Failed: 120
Progress: 300/500 LSOAs | Success: 1060 | Failed: 140
Progress: 350/500 LSOAs | Success: 1220 | Failed: 180
Progress: 400/500 LSOAs | Success: 1384 | Failed: 216
Progress: 450/500 LSOAs | Success: 1556 | Failed: 244
Progress: 500/500 LSOAs | Success: 1708 | Failed: 292

Done! 1708 downloaded, 292 failed
